In [12]:
import getpass
import os

password = getpass.getpass()
command = "sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null"
os.system('echo %s | %s' % (password, command))

 ·······


0

In [18]:
!sudo apt-get -y install openjdk-8-jre-headless
!pip install pyspark


Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u265-b01-0+deb9u1).
openjdk-8-jre-headless set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.
     |████████████████████████████████| 204.7 MB 16 kB/s s eta 0:00:01
     |████████████████████████████████| 198 kB 75.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=36fd05a2d0f1a3a372e51bdfbea19f17a942997e88137dc2e3ab9b2c35cf1f1c
  Stored in directory: /home/jupyter/.cache/pip/wheels/4e/c5/36/aef1bb711963a619063119cc032176106827a129c0be20e301
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

In [2]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as F

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS #, Rating #, MatrixFactorizationModel

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
# Visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(18, 4))

import seaborn as sns
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (18,4)})

np.set_printoptions(precision=4, suppress=True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
# set random seed for reproducability
rnd_seed=42
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

In [5]:
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("movie-recommender-pyspark")
         .getOrCreate())
spark

In [6]:
sc = spark.sparkContext
sc
print (sc.defaultParallelism)

<SparkContext master=local appName=pyspark-shell>

1


In [7]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

In [8]:
print(sc.getConf().toDebugString())

spark.app.id=local-1598274044823
spark.app.name=pyspark-shell
spark.driver.host=python-20200612-104541.us-west1-b.c.winter-agency-280010.internal
spark.driver.port=40945
spark.executor.id=driver
spark.master=local
spark.rdd.compress=True
spark.serializer.objectStreamReset=100
spark.submit.deployMode=client
spark.submit.pyFiles=
spark.ui.showConsoleProgress=true


In [9]:
train_features = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("mode", "DROPMALFORMED").load("train_features.csv")

In [10]:
test_features = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("mode", "DROPMALFORMED").load("test_features.csv")

In [11]:
test_features.limit(10).toPandas()

,_c0,Unnamed: 0,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length_x,genre_ids_x,artist_name_x,composer_x,lyricist_x,language_x,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length_y,genre_ids_y,artist_name_y,composer_y,lyricist_y,language_y,repeat_count,play_count,repeat_percentage,repeat_count_artist,play_count_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno,song_year
0,0,0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,None,3.0,愛其實很殘忍,TWUM71400047,1,0,None,7,2016-02-19,2017-09-18,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,None,3.0,350,694,50.4,7102.0,13654,52.0,142,36.6,2.0
1,1,1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,320470.0,465,林俊傑 (JJ Lin),林俊傑,孫燕姿/易家揚,3.0,她說,TWB671005201,1,0,None,7,2016-02-19,2017-09-18,320470.0,465,林俊傑 (JJ Lin),林俊傑,孫燕姿/易家揚,3.0,3804,6090,62.5,67621.0,115325,52.0,142,36.6,2.0
2,2,2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,None,song-based-playlist,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,Yu Takahashi,17.0,subarashiki nichijo,JPWP01070260,1,0,None,4,2016-11-17,2016-11-24,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,Yu Takahashi,17.0,2,5,40.0,286.0,989,19.0,132,14.4,2.0
3,3,3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,None,52.0,Hold Me| Thrill Me| Kiss Me| Kill Me,GBAAN0201228,3,30,male,9,2007-07-25,2017-04-30,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,None,52.0,7,31,22.6,228.0,698,115.0,388,29.6,0.0
4,4,4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,197590.0,873,Yoga Mr Sound,Neuromancer,None,-1.0,Om Yoga,ITO101121898,3,30,male,9,2007-07-25,2017-04-30,197590.0,873,Yoga Mr Sound,Neuromancer,None,-1.0,2,5,40.0,59.0,180,115.0,388,29.6,2.0
5,5,5,5,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,NV3nhEcMqsawwvSNTUAt9IVAexHLOm0lDfrHyEfN5B0=,radio,Radio,radio,216372.0,465,LEE HI,None,None,31.0,1|2|3|4,TWA531300295,3,30,male,9,2007-07-25,2017-04-30,216372.0,465,LEE HI,None,None,31.0,32,81,39.5,1895.0,3733,115.0,388,29.6,2.0
6,6,6,6,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,DPM6G9RB5QO2dvAVPyc70gxHdvu872IjTUvV6LJT8ho=,radio,Radio,radio,200306.0,465,BoA,Joe Belmaati,BoA / Joe Belmaati,31.0,The Shadow,KRA301200363,3,30,male,9,2007-07-25,2017-04-30,200306.0,465,BoA,Joe Belmaati,BoA / Joe Belmaati,31.0,34,90,37.8,1804.0,4425,115.0,388,29.6,2.0
7,7,7,7,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,JGXB3PHc0CX0JskwwjmYP8i318BLo7DhChgEj6Yqjt8=,my library,Local playlist more,local-playlist,344305.0,458,周傳雄 (Steve Chou),周傳雄,陳信榮,3.0,黃昏,TWA470053008,3,30,male,9,2007-07-25,2017-04-30,344305.0,458,周傳雄 (Steve Chou),周傳雄,陳信榮,3.0,628,1053,59.6,2235.0,4204,115.0,388,29.6,0.0
8,8,8,8,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,507plIkmke1jh3wMrHqKore82pPFozADwydR8P0Gx2Q=,radio,Radio,radio,289332.0,465,嚴藝丹 (Ivy Yan),嚴藝丹,嚴藝丹,3.0,三寸天堂,CNR211200331,3,30,male,9,2007-07-25,2017-04-30,289332.0,465,嚴藝丹 (Ivy Yan),嚴藝丹,嚴藝丹,3.0,546,848,64.4,878.0,1485,115.0,388,29.6,2.0
9,9,9,9,3ZQ6oGfcSiUoCrtBPKGa8hHCiFh5jqtDqPVDUl/zrjU=,HsgJXv1C7iVZiP7CWwWTfsmMhia6Huc0MUccfj+D02o=,my library,Local playlist more,local-library,232176.0,465,BIGBANG,None,None,31.0,FANTASTIC BABY- (冠軍曲),SGA051200112,1,0,None,7,2015-12-19,2017-09-18,232176.0,465,BIGBANG,None,None,31.0,1424,2381,59.8,35317.0,61927,224.0,313,71.6,2.0


In [12]:
test_features.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- msno: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- source_system_tab: string (nullable = true)
 |-- source_screen_name: string (nullable = true)
 |-- source_type: string (nullable = true)
 |-- song_length_x: double (nullable = true)
 |-- genre_ids_x: string (nullable = true)
 |-- artist_name_x: string (nullable = true)
 |-- composer_x: string (nullable = true)
 |-- lyricist_x: string (nullable = true)
 |-- language_x: double (nullable = true)
 |-- name: string (nullable = true)
 |-- isrc: string (nullable = true)
 |-- city: integer (nullable = true)
 |-- bd: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: integer (nullable = true)
 |-- registration_init_time: string (nullable = true)
 |-- expiration_date: string (nullable = true)
 |-- song_length_y: double (nullable = true)
 |-- genre_ids_y: string (null

In [20]:
test_features = test_features.drop("_c0", "Unnamed: 0", "language_y", "lyricist_y", "composer_y", "artist_name_y", "genre_ids_y",
                  "song_length_y")

In [21]:
(print("Number of songs with at least one rating strictly higher than 1000: {0}".format(test_features.filter('play_count > 1000')
                                                                                     .select('song_id')
                                                                                     .distinct()
                                                                                     .count())))

Number of songs with at least one rating strictly higher than 1000: 1072


In [22]:

# lm = (sns.lmplot(x='msno', y='song_id', hue='play_count', 
#                  data=test_features.toPandas(), 
#                  fit_reg=False, size=10, aspect=2, 
#                  palette=sns.palplot(sns.diverging_palette(250, 15, n=5, center="dark"))))

In [23]:
#因为训练集太大太大，难以训练
train_df, test_df = test_features.randomSplit([0.8, 0.2], seed=rnd_seed)

In [24]:

# we will cache the data because we will be using this dataframes over and over again
train_df.cache()
test_df.cache()

DataFrame[id: int, msno: string, song_id: string, source_system_tab: string, source_screen_name: string, source_type: string, song_length_x: double, genre_ids_x: string, artist_name_x: string, composer_x: string, lyricist_x: string, language_x: double, name: string, isrc: string, city: int, bd: int, gender: string, registered_via: int, registration_init_time: string, expiration_date: string, repeat_count: int, play_count: int, repeat_percentage: double, repeat_count_artist: double, play_count_artist: int, repeat_count_msno: double, play_count_msno: int, repeat_percentage_msno: double, song_year: double]

DataFrame[id: int, msno: string, song_id: string, source_system_tab: string, source_screen_name: string, source_type: string, song_length_x: double, genre_ids_x: string, artist_name_x: string, composer_x: string, lyricist_x: string, language_x: double, name: string, isrc: string, city: int, bd: int, gender: string, registered_via: int, registration_init_time: string, expiration_date: string, repeat_count: int, play_count: int, repeat_percentage: double, repeat_count_artist: double, play_count_artist: int, repeat_count_msno: double, play_count_msno: int, repeat_percentage_msno: double, song_year: double]

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(rank=10, maxIter=10, kagregParam=0.01, userCol='msno', itemCol='movie_id', ratingCol='target', coldStartStrategy="drop", seed=rnd_seed)

